In [1]:

%load_ext autoreload
%autoreload 2
import os
import random

import seaborn as sns
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import scipy

import tensorflow as tf
import tensorflow_datasets as tfds
import keras
from keras import layers, optimizers, losses, metrics, callbacks, ops
from PIL import Image
gpus = tf.config.experimental.list_physical_devices(device_type="GPU")
for gpu in gpus:
    tf.config.experimental.set_memory_growth(gpu, True)
plt.style.use("seaborn-v0_8")
random.seed(0)
np.random.seed(0)
tf.random.set_seed(0)
DATASET_PATH = "/mnt/dl/datasets/Oxford102FlowersSplits/"
os.environ["KERAS_BACKEND"] = "tensorflow"
LABELS = {i: k.strip() for i, k in enumerate(open(os.path.join(DATASET_PATH, "names.txt")))}
batch_size = 64
img_size = 224
SIZE = 128
batch_size = 32
num_classes = len(LABELS)
patch_size = 16
num_patches = img_size ** 2 / patch_size **2

2024-01-14 15:46:21.943173: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
/env_dl/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2024-01-14 15:46:24.065950: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-01-14 15:46:24.088751: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] succe

In [2]:
def load_dataset(split):
    
    def load_img(img_fname):
        img_bytes = tf.io.read_file(img_fname)
        img = tf.io.decode_jpeg(img_bytes)
        img = tf.image.resize(img, (img_size, img_size))
        img = tf.cast(img, tf.float32)
        return img
        
    path = os.path.join(DATASET_PATH, split, )
    img_files = os.listdir(os.path.join(path, "jpeg"))
    img_files = sorted(img_files, key=lambda x: int(x.replace(".jpeg", "")))
    img_files = list(img_files)[:SIZE]
    
    labels = list(open(os.path.join(path, "label", "label.txt"),))
    labels = [int(l.strip()) for l in labels][:SIZE]
    
    img_files = [os.path.join(path, "jpeg", name) for name in img_files]
    
    img_ds = tf.data.Dataset.from_tensor_slices(img_files).map(load_img).cache()
    label_ds = tf.data.Dataset.from_tensor_slices(labels).cache()
    ds = tf.data.Dataset.zip((img_ds, label_ds))
    return ds

In [3]:
train_ds = load_dataset("train")
validation_ds = load_dataset("validation")
test_ds = load_dataset("test")

2024-01-14 15:46:24.226129: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-01-14 15:46:24.226370: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-01-14 15:46:24.226519: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysf

In [4]:
aug_layers = [layers.RandomRotation(0.1), layers.RandomFlip()]

def preprocess(img, label, training):
    if training:
        for aug in aug_layers:
            img = aug(img)
    return tf.cast(img, tf.float32), label
    
train_ds = train_ds.shuffle(buffer_size=2048, seed=0).map(lambda img, label: preprocess(img, label, training=True)).batch(batch_size)
validation_ds = validation_ds.map(lambda img, label: preprocess(img, label, training=False), num_parallel_calls=5).batch(batch_size)
test_ds = test_ds.map(lambda img, label: preprocess(img, label, training=False), num_parallel_calls=5).batch(batch_size)

## Normalizing Flow

In [5]:
class Actnorm(layers.Layer):
    def __init__(self, **kwargs):
        super().__init__(**kwargs)
    
    def build(self, input_shape):
        h, w, c = input_shape[1:]
        self.hw = h * w
        
        self.s = self.add_variable(shape=(1, 1, 1, c,), 
                                   initializer=keras.initializers.glorot_uniform(),
                                   name="s",
                                   trainable=True)
        self.b = self.add_variable(shape=(c,), 
                                   initializer=keras.initializers.glorot_uniform(),
                                   name="b",
                                   trainable=True)
        return super().build(input_shape)
    
    def forward(self, x):
        return x * self.s + self.b
    
    def reverse(self, y):
        return ops.divide(y - self.b, self.s)
    
    def call(self, x, reverse=False):
        if not reverse:
            y = self.forward(x)
        else:
            y = self.reverse(x)
        log_det =  self.hw * ops.sum(ops.log(ops.abs(self.s)))
        return y, log_det
    
# Actnorm()(input)
# Actnorm()(np.random.normal(size=(2, 16, 16, 3)))


In [18]:
class Invertible1x1Conv(layers.Layer):
    
    def __init__(self, **kwargs):
        super().__init__(**kwargs)
    
    def build(self, input_shape):
        h, w, c = input_shape[1:]
        self.hw = h * w
        weight,  *_ = np.linalg.qr(np.random.normal(size=(c, c),).astype(np.float32))
        P, L, U = scipy.linalg.lu(weight)

        u_diag = np.diagonal(U)
        s = np.eye(c, dtype=np.float32)
        s = s * u_diag
        U -= s
        self.P = P
        
        self.L = self.add_variable(shape=(c, c),
                                   initializer=lambda args, dtype: tf.convert_to_tensor(L, dtype=tf.float32),
                                   trainable=True,
                                   dtype=tf.float32
                                   )
        
        self.l_mask = np.tril(np.ones((c, c), dtype=np.float32), -1)
        self.u_mask = np.triu(np.ones((c, c), dtype=np.float32), 1)
        self.diag_mask = np.eye(c, dtype=np.float32)
        
        self.U = self.add_variable(shape=(c, c),
                                   initializer=lambda args, dtype: tf.convert_to_tensor(U, dtype=tf.float32),
                                   trainable=True,
                                   dtype=tf.float32
                                   )

        self.s = self.add_variable(shape=(c, c),
                                   initializer=lambda args, dtype: tf.convert_to_tensor(s, dtype=tf.float32),
                                   trainable=True,
                                   dtype=tf.float32
                                   )
        super().build(input_shape)
    
    def forward(self, x, W):
        cin, cout = W.shape
        filters = tf.reshape(W, (1, 1, cin, cout))
        return tf.nn.conv2d(x, filters, (1, 1, 1, 1), padding="SAME")
    
    def reverse(self, y, W):
        cin, cout = W.shape
        w_inv = tf.linalg.inv(W)
        filters = tf.reshape(w_inv, (1, 1, cin, cout))
        return tf.nn.conv2d(y, filters, (1, 1, 1, 1), padding="SAME")
    
    def get_weights(self):
        P = self.P
        L = self.L * self.l_mask + self.diag_mask
        U = tf.multiply(self.U, self.u_mask)
        s = tf.multiply(self.s, self.diag_mask)
        W  =  P @ L @ (U + s)
        return W
    
    def call(self, x, reverse=False):
        W = self.get_weights()
        if not reverse:
            y = self.forward(x, W)
        else:
            y = self.reverse(x, W)
        return y, self.hw #*  tf.abs(tf.linalg.det(W))
# Invertible1x1Conv()(np.random.normal(size=(2, 4, 4, 5)))

In [19]:
class AffineCouplingLayer(layers.Layer):
    def __init__(self, dim=512, **kwargs):
        super().__init__(**kwargs)
        self.dim = dim
    def build(self, input_shape):
        c = input_shape[-1]
        self.nn = keras.Sequential([
            layers.Conv2D(self.dim, 3, padding="SAME"),
            layers.Activation("relu"),
            layers.Conv2D(self.dim, 1),
            layers.Activation("relu"),
            layers.Conv2D(c, 3, padding="SAME", 
                        #   kernel_initializer=keras.initializers.zeros(),
                        #   bias_initializer=keras.initializers.zeros(),
                          ),
        ])
    
        super().build(input_shape)

    
    def split(self, x):
        c = x.shape[-1]
        # s = c // 2
        xa = ops.take(x, range(0, c//2), axis=-1)
        xb = ops.take(x, range(c // 2, c), axis=-1)
        return xa, xb
    
    def concat(self, *args):
        return ops.concatenate(args, axis=-1)
    
    def forward(self, x):
        xa, xb = self.split(x)
        nn_out = self.nn(xb)
        log_s, t = self.split(nn_out)
        s = ops.exp(log_s)
        ya = s * xa + t
        yb = xb
        y = self.concat(ya, yb)
        
        return y, s
    
    def reverse(self, y):
        ya, yb = self.split(y)
        nn_out = self.nn(yb)
        log_s, t = self.split(nn_out)
        s = ops.exp(log_s)
        xa = (ya - t) / s
        xb = yb
        x = self.concat(xa, xb)
        return x, s
    
    def call(self, x, reverse=False):
        bz, *_ = ops.shape(x)
        if not reverse:
            y, s = self.forward(x)
        else:
            y, s = self.reverse(x)
    
        return  y, ops.sum(ops.reshape(ops.log(ops.abs(s)), (bz, -1)), -1)

# AffineCouplingLayer()(np.random.normal(size=(2, 4, 4, 6)))

In [20]:
class FlowStep(layers.Layer):
    def __init__(self, **kwargs):
        super().__init__(**kwargs)
        
    def build(self, input_shape):
        self.act_norm = Actnorm()
        self.inv_conv = Invertible1x1Conv()
        self.affine_coupling = AffineCouplingLayer()
        
    def call(self, x, reverse=False):
        if not reverse:
            y, log_det = self.forward(x)
        else:
            y, log_det = self.reverse(x)
        
        return y, log_det
    
    def forward(self, x):
        x, log_det1 = self.act_norm(x, reverse=False)
        x, log_det2 = self.inv_conv(x, reverse=False)
        x, log_det3 = self.affine_coupling(x, reverse=False)

        return x, log_det1 + log_det2 + log_det3
        
    
    def reverse(self, x):
        x, log_det3 = self.affine_coupling(x, reverse=True)
        x, log_det2 = self.inv_conv(x, reverse=True)
        x, log_det1 = self.act_norm(x, reverse=True)
        
        return x, log_det1 + log_det2 + log_det3

# FlowStep()(np.random.normal(size=(10, 400, 400, 6)))

In [21]:
class Squeeze(layers.Layer):
    
    def call(self, x, reverse=False):
        if not reverse:
            return self.forward(x)
        else:
            return self.reverse(x)
    
    def forward(self, x):
        bz, h, w, c = ops.shape(x)
        sqz_x = ops.reshape(x, (bz, h//2, 2, w//2, 2, c))
        sqz_x = ops.transpose(sqz_x, (0, 1, 3, 2, 4, 5))
        sqz_x = ops.reshape(sqz_x, tuple(ops.shape(sqz_x)[:3]) + (4*c, ))
        return sqz_x

    def reverse(self, x):
        bz, h, w, c = ops.shape(x)
        unsqz_x = ops.reshape(x, (bz, h, w, 2, 2, c // 4))
        unsqz_x = ops.transpose(unsqz_x, (0, 1, 3, 2, 4, 5))
        unsqz_x = ops.reshape(unsqz_x, (bz, 2*h, 2*w, unsqz_x.shape[-1]))
        return unsqz_x        
        
    
# Squeeze()(np.arange(2* 40 * 60 * 3).reshape((2, 40, 60, 3)))
        

In [22]:
def gaussian_logp(x, mean, log_std):
    return -0.5 * ops.log(2 * np.pi) - log_std - 0.5 * ops.square(x - mean) / ops.exp(2*log_std)

def gaussian_sample(mean, log_std, eps=None):
    if eps is None:
        eps = tf.random.normal(ops.shape(log_std))
    return mean + ops.exp(log_std) * eps

In [23]:
class Split(layers.Layer):
    
    def __init__(self,  split=True, **kwargs):
        super().__init__(**kwargs)
        self._split = split
        
    def build(self, input_shape):
        h, w, c = input_shape[1:]
        nc = c if self._split else  2*c
        self.conv_net = layers.Conv2D(nc, 3, padding="SAME")
    
    def call(self, x, reverse=False):
        if not reverse:
            y, z, log_p = self.forward(x)
        else:
            y, z, log_p = self.reverse(x)
            
        return y, z, log_p 

    def forward(self, x):
        bz, *_ = ops.shape(x)
        if self._split:
            out, z = ops.split(x, 2, axis=-1)
            mean, log_std = ops.split(self.conv_net(out), 2, -1)
        else:
            zeros = ops.zeros_like(x)
            z, out = x, x
            mean, log_std =  ops.split(self.conv_net(zeros), 2, -1)
            
        log_p = gaussian_logp(z, mean, log_std)
        log_p = ops.sum(ops.reshape(log_p, (bz, -1)), -1)
        return out, z, log_p
    
    def reverse(self, z):
        if self._split:
            mean, log_std =  ops.split(self.conv_net(z), 2, -1)
            z_sample = gaussian_sample(mean, log_std)
            z_sample = ops.concatenate([z, z_sample], axis=-1)
        else:
            zeros = ops.zeros_like(z)
            mean, log_std =  ops.split(self.conv_net(zeros), 2, -1)
            # TODO
            z_sample = gaussian_sample(mean, log_std)
        
        return z_sample, None, None
                

In [24]:
class Block(layers.Layer):
    
    def __init__(self, flow_steps, channel, split=True, **kwargs):
        super().__init__(**kwargs)
        self.flow_steps = flow_steps
        self.channel = channel
        self._split = split
    
    def build(self, input_shape):
        self.squeeze= Squeeze()
        self.flow_step = [FlowStep() for _ in range(self.flow_steps)]
        self.split = Split(self._split) 
            
    def call(self, x, reverse=False):
        if not reverse:
            return self.forward(x)
        else:
            return self.reverse(x)
    
    def forward(self, x):
        out = self.squeeze(x, reverse=False)
        log_det = 0.
        for i in range(self.flow_steps):
            out, flow_log_det = self.flow_step[i](out, reverse=False)
            
            log_det = log_det + flow_log_det
        out, z, log_p = self.split(out, reverse=False)
        return out, z, log_p, log_det

    def reverse(self, z):
        z, *_ = self.split(z, reverse=True)
        for i in reversed(range(self.flow_steps)):
            z, *_ = self.flow_step[i](z, reverse=True)
        z = self.squeeze(z, reverse=True)
        return z, None, None, None
        

In [88]:
class NFModel(keras.Model):
    
    def __init__(self, flow_steps, num_layers,**kwargs):
        super().__init__(**kwargs)
        self.flow_steps = flow_steps
        self.num_layers = num_layers
    
    def build(self, input_shape):
        h, w, c = input_shape[1:]
        channels = [np.power(2, i) * 3 for i in range(self.num_layers)]
        self.z_shapes = [(h // np.power(2, i), w // np.power(2, i), 
                          ( 6 * 2**i if i == self.num_layers else 3*2**i)) for i in range(self.num_layers, 0, -1)]
        self.blocks = [
            Block(self.flow_steps, channels[i], 
                  split= i != (self.num_layers - 1)) 
            for i in range(self.num_layers)
        ]
        return super().build(input_shape)
        
    def compile(self, optimizer: optimizers.Optimizer):
        self.optimizer = optimizer
        return super().compile(loss=None)
    
    def train_step(self, x):
        img, _ = x
        
        with tf.GradientTape() as tape:
            out, z_list, log_p, log_det = self(img, reverse=False)
            losses = -tf.reduce_mean(log_p + log_det)
        # TODO
        if tf.math.is_nan(losses):
            losses = 0.0
        else:
            grads = tape.gradient(losses, self.trainable_variables)
            clipped_grads = [tf.clip_by_norm(g, 5.) for g in grads]
            self.optimizer.apply_gradients(zip(clipped_grads, self.trainable_variables))
            self.add_loss(losses)
            return {"loss": tf.constant(0.)}
        
    def compute_loss(self,  x=None, y=None, y_pred=None, sample_weight=None, allow_empty=False ):
        return 0.0
    
    def call(self, x, reverse=False):
        if not reverse:
            return self.forward(x)
        else:
            return self.reverse(x)
            
    def forward(self, x):
        out = x 
        z_list = []
        log_det = 0.
        log_p = 0.
        for block_fn in self.blocks:
            out, z, block_log_p, block_log_det = block_fn(out, reverse=False)
            log_p = log_p + block_log_p
            log_det = log_det + block_log_det
            z_list.append(z)

        return out, z_list, log_p, log_det

    def reverse(self, z):
        for i, block_fn in enumerate(self.blocks[::-1]):
            z, *_ = block_fn(z, reverse=True)
        return z

In [89]:
input = keras.Input((img_size, img_size, 3))
nf_model = NFModel(2, 3)
nf_model(input)

(<KerasTensor shape=(None, 28, 28, 48), dtype=float32, sparse=False, name=keras_tensor_596>,
 [<KerasTensor shape=(None, 112, 112, 6), dtype=float32, sparse=False, name=keras_tensor_597>,
  <KerasTensor shape=(None, 56, 56, 12), dtype=float32, sparse=False, name=keras_tensor_598>,
  <KerasTensor shape=(None, 28, 28, 48), dtype=float32, sparse=False, name=keras_tensor_599>],
 <KerasTensor shape=(None,), dtype=float32, sparse=False, name=keras_tensor_600>,
 <KerasTensor shape=(None,), dtype=float32, sparse=False, name=keras_tensor_601>)

In [90]:
nf_model.summary()

Model: "nf_model_13"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape              ┃    Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━┩
│ block_39 (Block)                │ ?                         │    693,820 │
├─────────────────────────────────┼───────────────────────────┼────────────┤
│ block_40 (Block)                │ ?                         │    864,328 │
├─────────────────────────────────┼───────────────────────────┼────────────┤
│ block_41 (Block)                │ ?                         │  1,245,568 │
└─────────────────────────────────┴───────────────────────────┴────────────┘

 Total params: 2,803,716 (10.70 MB)

 Trainable params: 2,803,716 (10.70 MB)

 Non-trainable params: 0 (0.00 B)

In [91]:
nf_model.compile(optimizer=keras.optimizers.Adam(learning_rate=1e-4))

In [92]:
nf_model.fit(train_ds, validation_data=validation_ds, epochs=3)

Epoch 1/3


AttributeError: 'Tensor' object has no attribute 'numpy'

In [ ]:
img, _ = next(iter(train_ds))

In [ ]:
tf.math.reduce_max(img)

In [ ]:
img = img[0:1]
img.shape

In [ ]:
img = img / 255.0

In [ ]:
tf.reduce_max(img)

In [ ]:
out, z_list, log_p, log_det = nf_model(img)

In [ ]:
out.shape

In [ ]:
log_det

In [ ]:
log_p

In [ ]:
rec_img = nf_model.reverse(out)

In [ ]:
rec_img.shape

In [ ]:
tf.math.reduce_max(rec_img)

In [ ]:
Image.fromarray(np.clip(img.numpy() * 255., a_min=0, a_max=255).astype(np.uint8)[0])